In [41]:
from bs4 import BeautifulSoup
import requests
from openai import OpenAI
import json
import jsonlines
import os
import sqlite3


In [10]:
judge_wikis = [
    "https://en.wikipedia.org/wiki/Julien_Neals",
    "https://en.wikipedia.org/wiki/Regina_M._Rodriguez",
    "https://en.wikipedia.org/wiki/Zahid_Quraishi",
    "https://en.wikipedia.org/wiki/Lydia_Griggsby",
    "https://en.wikipedia.org/wiki/Deborah_Boardman",
    "https://en.wikipedia.org/wiki/David_Estudillo",
    "https://en.wikipedia.org/wiki/Angel_Kelley",
    "https://en.wikipedia.org/wiki/Margaret_Strickland_(judge)",
    "https://en.wikipedia.org/wiki/Florence_Y._Pan",
    "https://en.wikipedia.org/wiki/Lauren_J._King",
    "https://en.wikipedia.org/wiki/Sarah_A._L._Merriam",
    "https://en.wikipedia.org/wiki/Christine_O%27Hearn",
    "https://en.wikipedia.org/wiki/Tana_Lin",
    "https://en.wikipedia.org/wiki/Jia_M._Cobb",
    "https://en.wikipedia.org/wiki/Karen_M._Williams",
    "https://en.wikipedia.org/wiki/Patricia_Tolliver_Giles",
    "https://en.wikipedia.org/wiki/Michael_S._Nachmanoff",
    "https://en.wikipedia.org/wiki/Sarala_Nagala",
    "https://en.wikipedia.org/wiki/Omar_A._Williams",
    "https://en.wikipedia.org/wiki/Samantha_D._Elliott",
    "https://en.wikipedia.org/wiki/Linda_Lopez_(judge)",
    "https://en.wikipedia.org/wiki/Jinsook_Ohta",
    "https://en.wikipedia.org/wiki/David_H._Urias",
    "https://en.wikipedia.org/wiki/Maame_Ewusi-Mensah_Frimpong",
    "https://en.wikipedia.org/wiki/Jane_M._Beckering",
    "https://en.wikipedia.org/wiki/Shalina_D._Kumar",
    "https://en.wikipedia.org/wiki/Jennifer_L._Thurston",
    "https://en.wikipedia.org/wiki/Katherine_M._Menendez",
    "https://en.wikipedia.org/wiki/Mary_Dimke",
    "https://en.wikipedia.org/wiki/Bridget_M._Brennan",
    "https://en.wikipedia.org/wiki/Charles_E._Fleming",
    "https://en.wikipedia.org/wiki/David_A._Ruiz",
    "https://en.wikipedia.org/wiki/Jacqueline_Scott_Corley",
    "https://en.wikipedia.org/wiki/Fred_W._Slaughter",
    "https://en.wikipedia.org/wiki/Ruth_Bermudez_Montenegro",
    "https://en.wikipedia.org/wiki/Victoria_Calvert",
    "https://en.wikipedia.org/wiki/Julie_Rubin",
    "https://en.wikipedia.org/wiki/Hector_Gonzalez_(judge)",
    "https://en.wikipedia.org/wiki/John_H._Chun",
    "https://en.wikipedia.org/wiki/Cristina_D._Silva",
    "https://en.wikipedia.org/wiki/Anne_Traum",
    "https://en.wikipedia.org/wiki/Sarah_Geraghty",
    "https://en.wikipedia.org/wiki/Georgette_Castner",
    "https://en.wikipedia.org/wiki/Sherilyn_Peace_Garnett",
    "https://en.wikipedia.org/wiki/Jennifer_L._Rochon",
    "https://en.wikipedia.org/wiki/Trina_Thompson",
    "https://en.wikipedia.org/wiki/Sunshine_Sykes",
    "https://en.wikipedia.org/wiki/Evelyn_Padin",
    "https://en.wikipedia.org/wiki/Charlotte_Sweeney",
    "https://en.wikipedia.org/wiki/Nina_Morrison_(judge)",
    "https://en.wikipedia.org/wiki/Robert_S._Huie",
    "https://en.wikipedia.org/wiki/Ana_de_Alba",
    "https://en.wikipedia.org/wiki/Stephen_H._Locher",
    "https://en.wikipedia.org/wiki/Nina_Y._Wang",
    "https://en.wikipedia.org/wiki/Nancy_L._Maldonado",
    "https://en.wikipedia.org/wiki/Gregory_B._Williams",
    "https://en.wikipedia.org/wiki/Elizabeth_Hanes",
    "https://en.wikipedia.org/wiki/Jennifer_H._Rearden",
    "https://en.wikipedia.org/wiki/Mar%C3%ADa_Antongiorgi-Jord%C3%A1n",
    "https://en.wikipedia.org/wiki/Camille_V%C3%A9lez-Riv%C3%A9",
    "https://en.wikipedia.org/wiki/Anne_M._Nardacci",
    "https://en.wikipedia.org/wiki/F._Kay_Behm",
    "https://en.wikipedia.org/wiki/Kelley_B._Hodge",
    "https://en.wikipedia.org/wiki/Mia_Roberts_Perez",
    "https://en.wikipedia.org/wiki/Kai_Scott",
    "https://en.wikipedia.org/wiki/John_Frank_Murphy",
    "https://en.wikipedia.org/wiki/Jerry_W._Blackwell",
    "https://en.wikipedia.org/wiki/Jeffery_P._Hopkins",
    "https://en.wikipedia.org/wiki/Gina_R._M%C3%A9ndez-Mir%C3%B3",
    "https://en.wikipedia.org/wiki/Lindsay_C._Jenkins",
    "https://en.wikipedia.org/wiki/Matthew_L._Garcia",
    "https://en.wikipedia.org/wiki/Adrienne_Nelson",
    "https://en.wikipedia.org/wiki/Ana_C._Reyes",
    "https://en.wikipedia.org/wiki/Daniel_Calabretta",
    "https://en.wikipedia.org/wiki/Jamar_K._Walker",
    "https://en.wikipedia.org/wiki/Jamal_Whitehead",
    "https://en.wikipedia.org/wiki/Araceli_Mart%C3%ADnez-Olgu%C3%ADn",
    "https://en.wikipedia.org/wiki/Margaret_R._Guzman",
    "https://en.wikipedia.org/wiki/Colleen_Lawless",
    "https://en.wikipedia.org/wiki/Jonathan_J._C._Grey",
    "https://en.wikipedia.org/wiki/Robert_S._Ballou",
    "https://en.wikipedia.org/wiki/Andrew_G._Schopler",
    "https://en.wikipedia.org/wiki/Arun_Subramanian",
    "https://en.wikipedia.org/wiki/James_E._Simmons_Jr.",
    "https://en.wikipedia.org/wiki/Jessica_G._L._Clarke",
    "https://en.wikipedia.org/wiki/Gordon_Gallagher",
    "https://en.wikipedia.org/wiki/Matthew_P._Brookman",
    "https://en.wikipedia.org/wiki/Michael_E._Farbiarz",
    "https://en.wikipedia.org/wiki/Robert_Kirsch_(judge)",
    "https://en.wikipedia.org/wiki/Orelia_Merchant",
    "https://en.wikipedia.org/wiki/Wesley_Hsu",
    "https://en.wikipedia.org/wiki/LaShonda_A._Hunt",
    "https://en.wikipedia.org/wiki/Amanda_Brailsford",
    "https://en.wikipedia.org/wiki/Jeremy_C._Daniel",
    "https://en.wikipedia.org/wiki/Darrel_J._Papillion",
    "https://en.wikipedia.org/wiki/Hern%C3%A1n_D._Vera",
    "https://en.wikipedia.org/wiki/P._Casey_Pitts",
    "https://en.wikipedia.org/wiki/Dale_Ho",
    "https://en.wikipedia.org/wiki/Nusrat_Jahan_Choudhury",
    "https://en.wikipedia.org/wiki/Natasha_C._Merle",
    "https://en.wikipedia.org/wiki/Kymberly_Evanson",
    "https://en.wikipedia.org/wiki/Tiffany_Cartwright",
    "https://en.wikipedia.org/wiki/Myong_J._Joun",
    "https://en.wikipedia.org/wiki/Jeffrey_Cummings",
    "https://en.wikipedia.org/wiki/Vernon_D._Oliver",
    "https://en.wikipedia.org/wiki/Rita_F._Lin",
    "https://en.wikipedia.org/wiki/Brendan_A._Hurson",
    "https://en.wikipedia.org/wiki/Susan_K._DeClercq",
    "https://en.wikipedia.org/wiki/Jennifer_L._Hall",
    "https://en.wikipedia.org/wiki/Julia_K._Munley",
    "https://en.wikipedia.org/wiki/Matthew_J._Maddox",
    "https://en.wikipedia.org/wiki/Kenly_Kiya_Kato",
    "https://en.wikipedia.org/wiki/Julia_Kobick",
    "https://en.wikipedia.org/wiki/Ramon_Reyes",
    "https://en.wikipedia.org/wiki/M%C3%B3nica_Ram%C3%ADrez_Almadani",
    "https://en.wikipedia.org/wiki/Brandy_R._McMillion",
    "https://en.wikipedia.org/wiki/Jeffrey_Bryan",
    "https://en.wikipedia.org/wiki/Margaret_Garnett",
    "https://en.wikipedia.org/wiki/Micah_W._J._Smith",
    "https://en.wikipedia.org/wiki/Jamel_K._Semper",
    "https://en.wikipedia.org/wiki/Shanlyn_A._S._Park",
    "https://en.wikipedia.org/wiki/Loren_AliKhan",
    "https://en.wikipedia.org/wiki/Jerry_Edwards_Jr.",
    "https://en.wikipedia.org/wiki/Brandon_Scott_Long",
    "https://en.wikipedia.org/wiki/Sara_E._Hill",
    "https://en.wikipedia.org/wiki/John_D._Russell_(judge)",
    "https://en.wikipedia.org/wiki/John_A._Kazen",
    "https://en.wikipedia.org/wiki/Kato_Crews",
    "https://en.wikipedia.org/wiki/Jacquelyn_D._Austin",
    "https://en.wikipedia.org/wiki/Cristal_C._Brisco",
    "https://en.wikipedia.org/wiki/Gretchen_S._Lund",
    "https://en.wikipedia.org/wiki/Kirk_E._Sherriff",
    "https://en.wikipedia.org/wiki/Karoline_Mehalchick",
    "https://en.wikipedia.org/wiki/Amy_M._Baggio",
    "https://en.wikipedia.org/wiki/Jacqueline_Becerra",
    "https://en.wikipedia.org/wiki/David_S._Leibowitz",
    "https://en.wikipedia.org/wiki/Julie_S._Sneed",
    "https://en.wikipedia.org/wiki/Melissa_Damian",
    "https://en.wikipedia.org/wiki/Kelly_H._Rankin",
    "https://en.wikipedia.org/wiki/Jasmine_H._Yoon",
    "https://en.wikipedia.org/wiki/Sunil_Harjani",
    "https://en.wikipedia.org/wiki/Melissa_R._DuBose",
    "https://en.wikipedia.org/wiki/Edward_S._Kiel",
    "https://en.wikipedia.org/wiki/Eumi_K._Lee",
    "https://en.wikipedia.org/wiki/Ernesto_Gonzalez_(judge)",
    "https://en.wikipedia.org/wiki/Leon_Schydlower",
    "https://en.wikipedia.org/wiki/Susan_M._Bazis",
    "https://en.wikipedia.org/wiki/Robert_J._White_(judge)",
    "https://en.wikipedia.org/wiki/Ann_Marie_McIff_Allen",
    "https://en.wikipedia.org/wiki/Georgia_N._Alexakis",
    "https://en.wikipedia.org/wiki/Sanket_J._Bulsara",
    "https://en.wikipedia.org/wiki/Eric_Schulte",
    "https://en.wikipedia.org/wiki/Camela_C._Theeler",
    "https://en.wikipedia.org/wiki/Krissa_M._Lanham",
    "https://en.wikipedia.org/wiki/Angela_M._Martinez",
    "https://en.wikipedia.org/wiki/Dena_M._Coggins",
    "https://en.wikipedia.org/wiki/Stacey_D._Neumann",
    "https://en.wikipedia.org/wiki/Meredith_Vacca",
    "https://en.wikipedia.org/wiki/Joseph_F._Saporito_Jr.",
    "https://en.wikipedia.org/wiki/Adam_B._Abelson",
    "https://en.wikipedia.org/wiki/Jeannette_Vargas",
    "https://en.wikipedia.org/wiki/Mary_Kay_Lanthier",
    "https://en.wikipedia.org/wiki/Laura_Provinzino",
    "https://en.wikipedia.org/wiki/Mary_Kay_Costello",
    "https://en.wikipedia.org/wiki/Michelle_Williams_Court",
    "https://en.wikipedia.org/wiki/Byron_B._Conway",
    "https://en.wikipedia.org/wiki/April_Perry",
    "https://en.wikipedia.org/wiki/Jonathan_E._Hawley",
    "https://en.wikipedia.org/wiki/Mustafa_T._Kasubhai",
    "https://en.wikipedia.org/wiki/Sarah_F._Russell",
    "https://en.wikipedia.org/wiki/Rebecca_L._Pennell",
    "https://en.wikipedia.org/wiki/Amir_Ali_(judge)",
    "https://en.wikipedia.org/wiki/Sharad_H._Desai",
    "https://en.wikipedia.org/wiki/Anne_Hwang",
    "https://en.wikipedia.org/wiki/Brian_E._Murphy",
    "https://en.wikipedia.org/wiki/Sparkle_L._Sooknanan",
    "https://en.wikipedia.org/wiki/Catherine_Henry",
    "https://en.wikipedia.org/wiki/Gail_A._Weilheimer",
    "https://en.wikipedia.org/wiki/Anthony_Brindisi",
    "https://en.wikipedia.org/wiki/Elizabeth_C._Coombe",
    "https://en.wikipedia.org/wiki/Sarah_M._Davenport",
    "https://en.wikipedia.org/wiki/Tiffany_R._Johnson",
    "https://en.wikipedia.org/wiki/Keli_M._Neary",
    "https://en.wikipedia.org/wiki/Cynthia_Valenzuela_Dixon",
    "https://en.wikipedia.org/wiki/No%C3%ABl_Wise"
]

In [27]:
def scrape_wiki(url):
    # Define a user agent
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    # Send a GET request to the webpage with the user agent
    response = requests.get(url, headers=headers)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the webpage content
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Extract the text from the div with id=bodyContent
        body_content = soup.find(id='bodyContent')
        if body_content:
            full_text = body_content.get_text()
            # Trim the text to only include content up to "References[edit]"
            trimmed_text = full_text.split("References[edit]")[0]
            return trimmed_text
        else:
            print("No content found with id 'bodyContent'.")
            return None
    else:
        print(f"Failed to retrieve the webpage. Status code: {response.status_code}")
        return None


client = OpenAI()

def extract_insights(wiki_page):
    system_prompt = """
    You are tasked with extracting data from a Wikipedia page. Your goal is to identify and extract key information about the person, organizing it into thematic groups or subgroups. Once the data is organized, output it in a structured JSON format.
    """
    response = client.chat.completions.create(
            model="gpt-4o",
            response_format={ "type": "json_object" },
            messages=[
                {
                    "role": "system",
                    "content": system_prompt
                },
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": "Extract the data in this wikipedia page and output into JSON."},
                        {"type": "text", "text": f"{wiki_page}"},
                    ]
                }
            ],
            temperature=0.0,
        )
    return response.choices[0].message.content

def append_to_jsonl(json_data, url):
    """
    Appends a JSON object to a JSON Lines file with indentation for readability.
    
    Args:
        json_data (dict): The JSON object to append.
        url (str): The URL of the Wikipedia page.
    """
    # Create the results directory if it doesn't exist
    os.makedirs('results', exist_ok=True)
    
    # Extract the name from the URL to use as the filename
    filename = url.split("https://en.wikipedia.org/wiki/")[-1] + ".jsonl"
    file_path = os.path.join('results', filename)
    
    with open(file_path, 'a') as file:
        json.dump(json_data, file, indent=4)
        file.write('\n')  # Ensure each JSON object is on a new line

# Process each URL
for url in judge_wikis:
    wiki_text = scrape_wiki(url)
    if wiki_text:
        insights = extract_insights(wiki_text)
        if insights:
            # Parse and append
            sample_json = json.loads(insights)
            append_to_jsonl(sample_json, url)


In [58]:
def merge_jsonl_files(directory='results', output_file='merged.jsonl'):
    """
    Merges all JSON Lines files from a directory into a single JSON Lines file.

    Args:
        directory (str): The directory containing the JSON Lines files to merge.
        output_file (str): The name of the output file.
    """
    files = [f for f in os.listdir(directory) if f.endswith('.jsonl')]

    with open(output_file, 'w') as outfile:
        for filename in files:
            file_path = os.path.join(directory, filename)
            with open(file_path, 'r') as infile:
                for line in infile:
                    outfile.write(line + '\n')
merge_jsonl_files()


In [61]:
# Reference Schema to Mold Raw JSONS to 
ref_schema = """[
    {
        "personal_details": {
            "name": "string",
            "birth_year": "integer",
            "birth_place": "string"
        },
        "education": [
            {
                "institution": "string",
                "degree": "string",
                "field_of_study": "string",
                "honors": "string",
                "year": "integer"
            }
        ],
        "career": {
            "positions": [
                {
                    "role": "string",
                    "organization": "string",
                    "position_title": "string",
                    "start_year": "integer",
                    "end_year": "integer",
                    "location": "string",
                    "details": "string"
                }
            ],
            "clerkships": [
                {
                    "judge": "string",
                    "court": "string",
                    "years": "string"  // Format: "YYYY-YYYY"
                }
            ],
            "private_practice": [
                {
                    "firm": "string",
                    "position": "string",
                    "years": "string", // Format: "YYYY-YYYY"
                    "focus_area": "string",
                    "notable_case": "string"
                }
            ],
            "academic_roles": [
                {
                    "role": "string",
                    "institution": "string",
                    "duration": "string", // Example: "2015-2020"
                    "department": "string"
                }
            ]
        }
    }
]
"""

In [62]:
def simulate_transformation(json_line, system_prompt):
    
    response = client.chat.completions.create(
        model="gpt-4o",
        response_format={ "type": "json_object" },
        messages=[
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": f"Transform the following raw JSON data according to the provided schema. Ensure all data is formatted as specified by values in the schema. Here is the raw JSON: {json_line}"}
                ]
            }
        ],
        temperature=0.0,
    )
    return json.loads(response.choices[0].message.content)

def transform_json_data(file_path, ref_schema):

    # system prompt with the reference schema
    system_prompt = f"""
    You are a data transformation tool that takes in JSON data and a reference JSON schema, and outputs JSON data according to the schema.
    Not all of the data in the input JSON will fit the schema, so you may need to omit some data or add null values to the output JSON.
    Ensure values are formatted as specified in the schema (e.g. dates as YYYY-MM-DD).
    Here is the schema:
    {ref_schema}
    """
    
    transformed_data = []
    
    # Load / transform each line of JSON 
    with jsonlines.open(file_path, 'r') as reader:
        for json_line in reader:
            transformed_data.append(simulate_transformation(json_line, system_prompt))
    
    # Write transformed data to new jsonl file in the 'transformed' directory
    os.makedirs('transformed', exist_ok=True)
    output_file_path = os.path.join('transformed', os.path.basename(file_path))
    
    with jsonlines.open(output_file_path, mode='w') as writer:
        writer.write_all(transformed_data)
    
    return transformed_data


In [ ]:
transform_json_data('merged.jsonl', ref_schema)

In [5]:
def load_data_to_sqlite(db_name, jsonl_file):
    # Connect to SQLite database (or create it if it doesn't exist)
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    
    # Create tables according to the new schema
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS personaldetails (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT,
            birth_year INTEGER,
            birth_place TEXT
        )
    ''')
    
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS education (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            personal_id INTEGER,
            institution TEXT,
            degree TEXT,
            field_of_study TEXT,
            honors TEXT,
            year INTEGER,
            FOREIGN KEY (personal_id) REFERENCES personaldetails (id)
        )
    ''')
    
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS career (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            personal_id INTEGER,
            FOREIGN KEY (personal_id) REFERENCES personaldetails (id)
        )
    ''')
    
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS positions (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            career_id INTEGER,
            role TEXT,
            organization TEXT,
            position_title TEXT,
            start_year INTEGER,
            end_year INTEGER,
            location TEXT,
            details TEXT,
            FOREIGN KEY (career_id) REFERENCES career (id)
        )
    ''')
    
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS clerkships (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            career_id INTEGER,
            judge TEXT,
            court TEXT,
            years TEXT,
            FOREIGN KEY (career_id) REFERENCES career (id)
        )
    ''')
    
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS privatepractice (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            career_id INTEGER,
            firm TEXT,
            "position" TEXT,
            years TEXT,
            focus_area TEXT,
            notable_case TEXT,
            FOREIGN KEY (career_id) REFERENCES career (id)
        )
    ''')
    
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS academicroles (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            career_id INTEGER,
            role TEXT,
            institution TEXT,
            duration TEXT,
            department TEXT,
            FOREIGN KEY (career_id) REFERENCES career (id)
        )
    ''')
    
    # Load in the JSONL data
    with jsonlines.open(jsonl_file, 'r') as reader:
        for obj in reader:
            # personal
            cursor.execute('''
                INSERT INTO personaldetails (name, birth_year, birth_place)
                VALUES (?, ?, ?)
            ''', (
                obj['personal_details']['name'],
                obj['personal_details']['birth_year'],
                obj['personal_details']['birth_place']
            ))
            personal_id = cursor.lastrowid
            
            # education
            for edu in obj.get('education', []):
                cursor.execute('''
                    INSERT INTO education (personal_id, institution, degree, field_of_study, honors, year)
                    VALUES (?, ?, ?, ?, ?, ?)
                ''', (
                    personal_id,
                    edu['institution'],
                    edu['degree'],
                    edu.get('field_of_study', None),
                    edu.get('honors', None),
                    edu['year']
                ))
            
            # career
            cursor.execute('''
                INSERT INTO career (personal_id)
                VALUES (?)
            ''', (personal_id,))
            career_id = cursor.lastrowid
            
            # positions
            for pos in obj['career'].get('positions', []):
                cursor.execute('''
                    INSERT INTO positions (career_id, role, organization, position_title, start_year, end_year, location, details)
                    VALUES (?, ?, ?, ?, ?, ?, ?, ?)
                ''', (
                    career_id,
                    pos.get('role', ''),
                    pos.get('organization', ''),
                    pos.get('position_title', ''),
                    pos['start_year'],
                    pos.get('end_year', None),
                    pos.get('location', ''),
                    pos.get('details', None)
                ))
            
            # clerkships
            for clerk in obj['career'].get('clerkships', []):
                cursor.execute('''
                    INSERT INTO clerkships (career_id, judge, court, years)
                    VALUES (?, ?, ?, ?)
                ''', (
                    career_id,
                    clerk['judge'],
                    clerk['court'],
                    clerk['years']
                ))
            
            # private practice
            for practice in obj['career'].get('private_practice', []):
                cursor.execute('''
                    INSERT INTO privatepractice (career_id, firm, "position", years, focus_area, notable_case)
                    VALUES (?, ?, ?, ?, ?, ?)
                ''', (
                    career_id,
                    practice['firm'],
                    practice['position'],
                    practice['years'],
                    practice.get('focus_area', None),
                    practice.get('notable_case', None)
                ))
            
            # academics
            for role in obj['career'].get('academic_roles', []):
                cursor.execute('''
                    INSERT INTO academicroles (career_id, role, institution, duration, department)
                    VALUES (?, ?, ?, ?, ?)
                ''', (
                    career_id,
                    role['role'],
                    role['institution'],
                    role['duration'],
                    role.get('department', None)
                ))
    
    # Commit and close
    conn.commit()
    conn.close()

# Load to SQLite database
load_data_to_sqlite('judges3.db', 'transformed/merged.jsonl')


In [ ]:
def get_roles_before_2021(db_name):
    # Connect to the SQLite database
    with sqlite3.connect(db_name) as conn:
        cursor = conn.cursor()
        
        # Query to find roles that had an end date before 2021
        query = '''
        SELECT pd.name, combined.role, combined.organization, combined.end_year
        FROM (
            SELECT p.role, p.organization, p.end_year, c.personal_id
            FROM positions p
            JOIN career c ON p.career_id = c.id
            WHERE p.role IS NOT NULL AND p.end_year IS NOT NULL AND p.end_year < 2021
            UNION ALL
            SELECT pp.position as role, pp.firm as organization, CAST(SUBSTR(pp.years, -4) AS INTEGER) as end_year, c.personal_id
            FROM privatepractice pp
            JOIN career c ON pp.career_id = c.id
            WHERE pp.position IS NOT NULL AND pp.years IS NOT NULL AND CAST(SUBSTR(pp.years, -4) AS INTEGER) < 2021
        ) AS combined
        JOIN personaldetails pd ON combined.personal_id = pd.id
        WHERE combined.role IS NOT NULL
        ORDER BY combined.end_year DESC
        '''
        
        cursor.execute(query)
        roles_before_2021 = cursor.fetchall()
        
    return roles_before_2021

# for readability
roles_before_2021 = get_roles_before_2021('judges3.db')
for role in roles_before_2021:
    print(f"Name: {role[0]}, \n Role: {role[1]}, \n Organization: {role[2]}, \nYear: {role[3]}\n\n")

In [ ]:
def get_partners_before_judges(db_name):
    with sqlite3.connect(db_name) as conn:
        cursor = conn.cursor()
        
        # Query to find where people were partners before becoming judges
        query = '''
        SELECT pd.name, pp.firm, pp.years
        FROM privatepractice pp
        JOIN career c ON pp.career_id = c.id
        JOIN personaldetails pd ON c.personal_id = pd.id
        WHERE pp.position LIKE '%Partner%'
        AND EXISTS (
            SELECT 1
            FROM positions p
            WHERE p.career_id = c.id
            AND p.role LIKE '%Judge%'
        )
        '''
        
        cursor.execute(query)
        partners_before_judges = cursor.fetchall()
        
    return partners_before_judges

# readable output
partners_before_judges = get_partners_before_judges('judges3.db')
for partner in partners_before_judges:
    print(f"Name: {partner[0]}, \n Firm: {partner[1]}, \n Years: {partner[2]}\n\n")